In [24]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

In [25]:
# ChromeDriver 경로 설정
service = Service("C:/chromedriver-win64/chromedriver.exe")
options = webdriver.ChromeOptions()

In [26]:
# 모바일 환경 설정
mobile_emulation = {
    "deviceMetrics": {"width": 360, "height": 640, "pixelRatio": 3.0},
    "userAgent": "Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1"
}
options.add_experimental_option("mobileEmulation", mobile_emulation)

In [27]:
# 브라우저 열기
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://app.catchtable.co.kr/ct/shop/sushikoji/review")  # 리뷰 페이지 URL

In [28]:
# 페이지 로드 대기
time.sleep(5)

In [29]:
# 무한 스크롤 처리
SCROLL_PAUSE_TIME = 3  # 스크롤 후 대기 시간
MAX_SCROLL = 200  # 최대 스크롤 반복 횟수 (2000개 리뷰 예상 기준)

scroll_count = 0
last_height = driver.execute_script("return document.body.scrollHeight")

while scroll_count < MAX_SCROLL:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 스크롤 끝까지 내리기
    time.sleep(SCROLL_PAUSE_TIME)

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # 더 이상 스크롤할 데이터가 없으면 종료
    last_height = new_height
    scroll_count += 1

In [30]:
# 리뷰 크롤링
reviews_data = []
unique_reviews = set()  # 중복 제거를 위한 Set

try:
    # 리뷰 컨테이너 찾기
    reviews = driver.find_elements(By.CLASS_NAME, "__my-review-post")  # 리뷰 컨테이너 클래스 이름
    for review in reviews:
        try:
            # 평점 추출
            rating = review.find_element(By.CLASS_NAME, "_10fm75h6").text

            # 리뷰어 이름 추출
            username = review.find_element(By.CLASS_NAME, "name.username").text

            # 리뷰 내용 추출
            content = review.find_element(By.CLASS_NAME, "review-content").text

            # 중복 여부 확인
            review_key = f"{username}_{content}"
            if review_key not in unique_reviews:
                unique_reviews.add(review_key)
                reviews_data.append({
                    "rating": rating,
                    "username": username,
                    "content": content
                })
        except Exception as e:
            print(f"리뷰 처리 중 에러 발생: {e}")

    # 리뷰 데이터를 CSV로 저장
    with open("sushikoji_reviews.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["rating", "username", "content"])
        writer.writeheader()  # 헤더 작성
        writer.writerows(reviews_data)  # 데이터 작성

    print(f"리뷰 데이터를 sushikoji_reviews.csv 파일에 저장했습니다! 총 {len(reviews_data)}개의 리뷰를 수집했습니다.")

finally:
    driver.quit()

리뷰 데이터를 sushikoji_reviews.csv 파일에 저장했습니다! 총 2074개의 리뷰를 수집했습니다.
